# Yield Prediction Model

In [37]:
import numpy as np
import pandas as pd
import joblib

df_yield = pd.read_csv('./district.csv')
df_yield.shape

(246091, 7)

In [38]:
df_yield = df_yield.rename(index=str, columns={"Production": 'Value'})
df_yield=df_yield.dropna()

In [39]:
features=df_yield.loc[:, df_yield.columns != 'Value']
label=df_yield['Value']

In [40]:
features = features.drop(["District_Name","Crop_Year"],axis=1)


In [41]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
transformed = ohe.fit_transform(features[["Season", "Crop", "State_Name"]])
transformed.shape
features_ohe =  np.hstack((transformed, features[['Area']].values ))
features_ohe.shape

(242361, 164)

In [42]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_ohe = scaler.fit_transform(features_ohe)
joblib.dump(scaler, 'yield_scaler.pkl')

['yield_scaler.pkl']

In [43]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(features_ohe, label, test_size=0.2, random_state=42)

In [44]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor()

In [45]:
RF.fit(train_data , train_labels)

RandomForestRegressor()

In [46]:
RF_accuracy = RF.score(test_data,test_labels)
RF_accuracy

0.8363661119044594

In [48]:
import joblib
joblib.dump(RF, 'RandomForestYieldPrediction.pkl')
joblib.dump(ohe, 'YieldOneHot.pkl')

['YieldOneHot.pkl']

# The Recommendation Model

In [41]:
reco_data = pd.read_csv('./matched_reco.csv')
reco_data.shape
reco_data = reco_data.drop(['Unnamed: 0'], axis=1)

In [42]:
reco_features=reco_data.loc[:, reco_data.columns != 'label']
reco_label=reco_data['label']
reco_features.tail()

N   P   K  temperature   humidity        ph    rainfall
2195  107  34  32    26.774637  66.413269  6.780064  177.774507
2196   99  15  27    27.417112  56.636362  6.086922  127.924610
2197  118  33  30    24.131797  67.225123  6.362608  173.322839
2198  117  32  34    26.272418  52.127394  6.758793  127.175293
2199  104  18  30    23.603016  60.396475  6.779833  140.937041

In [43]:
from sklearn.preprocessing import MinMaxScaler
reco_scaler=MinMaxScaler()
reco_features=reco_scaler.fit_transform(reco_features)

In [51]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
transformed_label = encoder.fit_transform(reco_label)

In [45]:
from sklearn.model_selection import train_test_split
reco_train_data, reco_test_data, reco_train_labels, reco_test_labels = train_test_split(reco_features, transformed_label, test_size=0.2, random_state=42)

In [46]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()

In [47]:
RFC.fit(reco_train_data , reco_train_labels)

RandomForestClassifier()

In [48]:
RFC_accuracy = RFC.score(reco_test_data,reco_test_labels)
RFC_accuracy

0.9931818181818182

In [52]:
import joblib
joblib.dump(RFC, 'RandomForestCropRecommendation.pkl')
joblib.dump(encoder, 'LabelEnc.pkl')
joblib.dump(reco_scaler, 'reco_scaler.pkl')

['reco_scaler.pkl']